# **Summer Analytics Capstone Project**

## **1. Setup and Library Installation**

In [ ]:
# Run this cell first in your Google Colab environment
!pip install pandas numpy pathway bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 12.0

## **2. Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import pathway as pw
from math import radians, sin, cos, sqrt, atan2

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column

## **3. Enable Bokeh to render plots in the notebook**

In [ ]:
output_notebook()

## **4. Data Loading and Preprocessing Function**

In [ ]:
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df["DateTime"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format='%d-%m-%Y %H:%M:%S')
    df = df.sort_values(by=["SystemCodeNumber", "DateTime"]).reset_index(drop=True)
    df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
    df = pd.get_dummies(df, columns=["VehicleType", "TrafficConditionNearby"], drop_first=True)
    return df

## **5. Pricing Models**
### **Model 1: Baseline Linear Model**

In [ ]:
# Model 1: Baseline Linear Model
def model_1_baseline_linear(df, alpha=0.1):
    initial_prices = {sys_code: 10.0 for sys_code in df["SystemCodeNumber"].unique()}
    df["Price_Model1"] = 0.0
    for sys_code in df["SystemCodeNumber"].unique():
        lot_df = df[df["SystemCodeNumber"] == sys_code].copy()
        prices = [initial_prices[sys_code]]
        for i in range(1, len(lot_df)):
            prev_price = prices[-1]
            occupancy_rate = lot_df.iloc[i]["OccupancyRate"]
            new_price = prev_price + alpha * occupancy_rate
            prices.append(new_price)
        df.loc[df["SystemCodeNumber"] == sys_code, "Price_Model1"] = prices
    return df

### **Model 2: Demand-Based Price Function**

In [ ]:
# Model 2: Demand-Based Price Function
def model_2_demand_based(df, base_price=10.0, lambda_val=0.5):
    alpha = 0.5
    beta = 0.2
    delta = 0.3
    vehicle_type_bike_weight = 0.1 if 'VehicleType_bike' in df.columns else 0
    vehicle_type_truck_weight = 0.2 if 'VehicleType_truck' in df.columns else 0
    vehicle_type_cycle_weight = 0.05 if 'VehicleType_cycle' in df.columns else 0
    traffic_high_weight = 0.1 if 'TrafficConditionNearby_high' in df.columns else 0
    traffic_low_weight = -0.1 if 'TrafficConditionNearby_low' in df.columns else 0

    df["Demand_Raw"] = (
        alpha * df["OccupancyRate"] +
        beta * df["QueueLength"] +
        delta * df["IsSpecialDay"] +
        vehicle_type_bike_weight * df.get('VehicleType_bike', 0) +
        vehicle_type_truck_weight * df.get('VehicleType_truck', 0) +
        vehicle_type_cycle_weight * df.get('VehicleType_cycle', 0) +
        traffic_high_weight * df.get('TrafficConditionNearby_high', 0) +
        traffic_low_weight * df.get('TrafficConditionNearby_low', 0)
    )
    min_demand = df["Demand_Raw"].min()
    max_demand = df["Demand_Raw"].max()
    df["Demand_Normalized"] = (df["Demand_Raw"] - min_demand) / (max_demand - min_demand)
    df["Price_Model2"] = base_price * (1 + lambda_val * df["Demand_Normalized"])
    df["Price_Model2"] = np.clip(df["Price_Model2"], 0.5 * base_price, 2.0 * base_price)
    return df

### **Model 3: Competitive Pricing Model**

In [ ]:
# Model 3: Competitive Pricing Model
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

def model_3_competitive_pricing(df, base_price=10.0, proximity_threshold=1.0):
    df["Price_Model3"] = df["Price_Model2"]
    unique_system_codes = df["SystemCodeNumber"].unique()
    locations = df[["SystemCodeNumber", "Latitude", "Longitude"]].drop_duplicates().set_index("SystemCodeNumber")

    for i, row in df.iterrows():
        current_sys_code = row["SystemCodeNumber"]
        current_lat = row["Latitude"]
        current_lon = row["Longitude"]
        current_occupancy_rate = row["OccupancyRate"]
        current_price = row["Price_Model2"]

        nearby_competitors = []
        for comp_sys_code in unique_system_codes:
            if comp_sys_code != current_sys_code:
                comp_lat = locations.loc[comp_sys_code]["Latitude"]
                comp_lon = locations.loc[comp_sys_code]["Longitude"]
                distance = haversine(current_lat, current_lon, comp_lat, comp_lon)

                if distance <= proximity_threshold:
                    competitor_price_at_time = df[(df["SystemCodeNumber"] == comp_sys_code) &
                                                  (df["DateTime"] == row["DateTime"])]

                    if not competitor_price_at_time.empty:
                        nearby_competitors.append({
                            "sys_code": comp_sys_code,
                            "price": competitor_price_at_time["Price_Model2"].iloc[0],
                            "occupancy_rate": competitor_price_at_time["OccupancyRate"].iloc[0]
                        })

        if nearby_competitors:
            avg_competitor_price = sum([comp["price"] for comp in nearby_competitors]) / len(nearby_competitors)

            if current_occupancy_rate > 0.9 and avg_competitor_price < current_price:
                df.loc[i, "Price_Model3"] = min(current_price, avg_competitor_price * 0.95)
            elif avg_competitor_price > current_price * 1.1:
                df.loc[i, "Price_Model3"] = current_price * 1.05
    return df

## **5. Pathway Real-Time Simulation (Conceptual Outline for Colab)**

This section demonstrates how Pathway would be integrated. Actual data streaming and real-time processing would require a running Pathway instance.

In [ ]:
# Actual data streaming and real-time processing would require a running Pathway instance.

# Function to simulate data stream (for demonstration)
def simulate_data_stream(df):
    for _, row in df.iterrows():
        yield row.to_dict()

# Main Pathway application logic
def run_pathway_app(data_source_path):
    # Ingest data from a CSV file as a real-time stream
    # For a true real-time scenario, this would be a Kafka, Redpanda, or other stream
    # Here, we simulate it by reading the CSV row by row

    # Create a Pathway input connector (conceptual)
    # data_stream = pw.io.csv.read(data_source_path, schema=None, mode=\'streaming\')

    # For local testing without a full Pathway setup, we'll use a simulated stream
    # In a real Pathway app, you'd use pw.io.csv.read or similar

    # Placeholder for Pathway data processing
    # In a real Pathway app, you'd apply your models here:
    # processed_stream = data_stream.apply(lambda row: preprocess_and_model(row))
    # prices_stream = processed_stream.apply(lambda row: calculate_prices(row))

    # For demonstration, we'll just use the pre-processed DataFrame
    print("Pathway simulation setup complete. In a real scenario, data would stream here.")
    print("Proceeding to visualize pre-calculated prices.")

## **6. Bokeh Visualisation**
This section demonstrates how Bokeh would be used for real-time visualization. In a true real-time scenario, the Bokeh plot would update dynamically with Pathway data.

In [ ]:
def visualize_prices(df):
    # Select a few parking lots for visualization to keep the plot readable
    sample_lots = df["SystemCodeNumber"].unique()[:3]

    plots = []
    for sys_code in sample_lots:
        lot_df = df[df["SystemCodeNumber"] == sys_code].copy()

        source = ColumnDataSource(data={
            'time': lot_df["DateTime"],
            'price_model1': lot_df["Price_Model1"],
            'price_model2': lot_df["Price_Model2"],
            'price_model3': lot_df["Price_Model3"]
        })

        p = figure(x_axis_type="datetime", title=f"Dynamic Pricing for {sys_code}", height=300, width=800)
        p.xaxis.axis_label = "Time"
        p.yaxis.axis_label = "Price ($)"

        p.line('time', 'price_model1', source=source, legend_label="Model 1", color="blue", line_width=2)
        p.line('time', 'price_model2', source=source, legend_label="Model 2", color="green", line_width=2)
        p.line('time', 'price_model3', source=source, legend_label="Model 3", color="red", line_width=2)

        p.legend.location = "top_left"
        p.legend.click_policy="hide"
        plots.append(p)

    show(column(*plots))

## **7. Main Execution Block of Collab**

In [ ]:
if __name__ == '__main__':
    # Upload your dataset.csv to your Colab environment or Google Drive
    # For example, if uploaded directly to Colab, the path would be:
    dataset_path = '/content/dataset.csv'
    # If from Google Drive, you'd need to mount Drive first:
    # from google.colab import drive
    # drive.mount('/content/drive')
    # dataset_path = '/content/drive/MyDrive/dataset.csv'

    print("Loading and preprocessing data...")
    processed_df = load_and_preprocess_data(dataset_path)
    print("Data preprocessing complete.")

    print("Applying Model 1...")
    processed_df = model_1_baseline_linear(processed_df)
    print("Applying Model 2...")
    processed_df = model_2_demand_based(processed_df)
    print("Applying Model 3 (this may take a while for large datasets)...")
    processed_df = model_3_competitive_pricing(processed_df)
    print("All models applied.")

    print("Running Pathway conceptual simulation...")
    run_pathway_app(dataset_path)

    print("Generating Bokeh visualizations...")
    visualize_prices(processed_df)

    print("Script execution complete. Please review the generated plots above.")

Loading and preprocessing data...
Data preprocessing complete.
Applying Model 1...
Applying Model 2...
Applying Model 3 (this may take a while for large datasets)...
All models applied.
Running Pathway conceptual simulation...
Pathway simulation setup complete. In a real scenario, data would stream here.
Proceeding to visualize pre-calculated prices.
Generating Bokeh visualizations...


Script execution complete. Please review the generated plots above.
